### 6️⃣ What are TOOLS?

Tools = let model call Python functions instead of guessing

🔥This kills hallucination 

OR

Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
A function or coroutine to execute.

In [15]:
from langchain_groq import ChatGroq
from langchain.tools import tool

# ✅ Original callable function
def _search_docs_fn(query: str):
    """Search documentation and return relevant information for a query."""
    return "Python was created by Guido van Rossum in 1991."

# ✅ Decorated tool for LLM
search_docs = tool(_search_docs_fn)

# Initialize LLM and bind the tool
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0
).bind_tools([search_docs])

# Ask the LLM to use the tool explicitly
response = llm.invoke("Use the search_docs tool to answer: Who created Python?")

# Check if the LLM wants to call a tool
if response.tool_calls:
    for call in response.tool_calls:
        tool_name = call["name"]
        args = call["args"]
        if tool_name == "search_docs":
            # ✅ Call the original function, not the StructuredTool
            tool_result = _search_docs_fn(**args)

            # Send tool output back to LLM
            final_answer = llm.invoke([
                response,
                {"role": "tool", "content": tool_result}
            ])
            print("Final Answer:", final_answer.content)
else:
    print("Final Answer:", response.content)
response


AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking "Who created Python?" I need to use the search_docs tool to find the answer. Let me check the available functions. There\'s a function called _search_docs_fn that takes a query parameter. The query here should be "Who created Python?" So I\'ll call that function with that exact query. I don\'t need to rephrase it. Just make sure the arguments are correctly formatted as JSON. Let me structure the tool_call accordingly.\n', 'tool_calls': [{'id': 'wgs9k5php', 'function': {'arguments': '{"query":"Who created Python?"}', 'name': '_search_docs_fn'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 162, 'total_tokens': 285, 'completion_time': 0.205454947, 'completion_tokens_details': {'reasoning_tokens': 95}, 'prompt_time': 0.006852069, 'prompt_tokens_details': None, 'queue_time': 0.055204811, 'total_time': 0.212307016}, 'model_name': 'qwen/qwen3-32b', 's

In [16]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response.content

'<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. I know they are birds, and some species like the African Grey Parrot are known for their ability to mimic human speech. But why do they do that? Is it just mimicry, or is there more to it?\n\nFirst, I should think about the basic reasons animals communicate. Communication in animals serves different purposes like attracting mates, warning of danger, or establishing territory. For parrots, talking might be part of their natural social behavior. Since they are social animals, maybe they use vocalizations to interact with each other. But why human speech?\n\nI remember reading that parrots have a specialized vocal organ called the syrinx, which allows them to produce a wide range of sounds. This might be more developed in parrots compared to other birds, enabling them to mimic human speech. But is there a purpose behind the mimicry?\n\nAnother angle is that in the wild, parrots l

In [17]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [19]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there\'s no typo. Everything looks good. Let me format the tool call correctly.\n', 'tool_calls': [{'id': 'pj3jrbvsm', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 154, 'total_tokens': 246, 'completion_time': 0.14727807, 'completion_tokens_details': {'reasoning_tokens': 68}, 'prompt_time': 0.006272983, 'prompt_tokens_details': None, 'queue_time': 0.055034408, 'total_time': 0.153551053}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run

### Tools Execution

In [20]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny. A perfect day to enjoy outdoor activities! 🌞


In [21]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. Since the user specified Boston, I need to call this function with "Boston" as the location. I\'ll make sure to format the tool call correctly within the XML tags as instructed.\n', 'tool_calls': [{'id': 'fjg89xv9h', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 153, 'total_tokens': 246, 'completion_time': 0.134275614, 'completion_tokens_details': {'reasoning_tokens': 69}, 'prompt_time': 0.005975489, 'prompt_tokens_details': None, 'queue_time': 0.050009238, 'total_time': 0.140251103}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_deman